#Data Augmentation


Data augmentation methods are essential for expanding datasets by generating synthetic data from existing ones.

While image augmentation is a well-established practice in computer vision, text augmentation is a relatively novel concept in the field of natural language processing (NLP).

The nlpaug module offers a range of efficient text augmentation algorithms capable of enhancing the performance of NLP models significantly.

In the previous section of this tutorial, we explored some exciting text augmentation functions available in the nlpaug module.

Now, in Part 2 , we will leverage the capabilities of the nlpaug module to produce text augmentations for Twitter tweets. Subsequently, we will assess the performance of bag-of-words models with and without employing text augmentations.

In [1]:
!pip install --upgrade gensim --quiet

In [2]:
import gensim
print(gensim.__version__)

4.3.2


In [3]:
!pip install transformers --quiet

In [4]:
import transformers

In [5]:
# Install the tokenizer needed by the back translation model
!pip install sacremoses --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 9.7 MB/s eta 0:00:00


In [6]:
# Install the tokenizer
import sacremoses

In [7]:
# Install the nlpaug module
!pip install nlpaug --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 7.9 MB/s eta 0:00:00


In [8]:

# Import the nlpaug module and its methods
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action

In [9]:

# Import other modules
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
import os
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [10]:

# Show all outputs of a cell in a jupyter notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#Twitter Climate Change Sentiment Dataset

The dataset can be downloaded from Kaggle.https://www.kaggle.com/datasets/edqian/twitter-climate-change-sentiment-dataset


The collection of this data was funded by a Canada Foundation for Innovation JELF Grant to Chris Bauch, University of Waterloo.


This dataset aggregates tweets pertaining to climate change collected between Apr 27, 2015 and Feb 21, 2018. In total, 43943 tweets were annotated. Each tweet is labelled independently by 3 reviewers. This dataset only contains tweets that all 3 reviewers agreed on (the rest were discarded).


Each tweet is labelled as one of the following classes:

2 (News): the tweet links to factual news about climate change
1 (Pro): the tweet supports the belief of man-made climate change
0 (Neutral): the tweet neither supports nor refutes the belief of man-made climate change
-1 (Anti): the tweet does not believe in man-made climate change

In [16]:
df = pd.read_csv("/content/twitter_sentiment_data.csv", encoding='utf-8')


# Rename column names and remove the tweetid column
df = df.rename(columns = {"sentiment": "label", "message": "text"}).drop('tweetid', axis = 1)

In [17]:
# Recode the labels to 0, 1, 2, and 3
# 0 - negative; 1 - neutral; 2 - positive; 3 - news
df['label'] = df['label'].replace([-1, 0, 1, 2],[0, 1, 2, 3])

In [18]:
# Important Note: Check the integrity of the DataFrame to ensure that there are no missing values,
# which will deter the training progress. Here, we simply drop any missing observations.
df = df.dropna()

In [19]:
# Take a look at the first five samples in the dataframe
pd.set_option('display.max_colwidth', None)
df.head()

,label,text
0,0,@tiniebeany climate change is an interesting hustle as it was global warming but the planet stopped warming for 15 yes while the suv boom
1,2,"RT @NatGeoChannel: Watch #BeforeTheFlood right here, as @LeoDiCaprio travels the world to tackle climate change https://t.co/LkDehj3tNn httÃ¢â‚¬Â¦"
2,2,Fabulous! Leonardo #DiCaprio's film on #climate change is brilliant!!! Do watch. https://t.co/7rV6BrmxjW via @youtube
3,2,RT @Mick_Fanning: Just watched this amazing documentary by leonardodicaprio on climate change. We all think thisÃ¢â‚¬Â¦ https://t.co/kNSTE8K8im
4,3,"RT @cnalive: Pranita Biswasi, a Lutheran from Odisha, gives testimony on effects of climate change &amp; natural disasters on the poÃ¢â‚¬Â¦"


#Tokenization

Tokenization is used in natural language processing to split paragraphs and sentences into smaller units that can be more easily assigned meaning.


The first step of the NLP process is gathering the data (a sentence) and breaking it into understandable parts (words).

In [20]:
# Initiate the NLTK word tokenizer
tokenizer = nltk.tokenize.TreebankWordTokenizer()

# Take a look at an example of tokenization
tokenizer.tokenize("I've been to Los Angeles before.")


['I', "'ve", 'been', 'to', 'Los', 'Angeles', 'before', '.']

#Stop Words

Stop words are a set of commonly used words in any language. For example, in English, “the”, “is” and “and”, would easily qualify as stop words.


In NLP and text mining applications, stop words are used to eliminate unimportant words, allowing applications to focus on the important words instead.

In [21]:

# Get a set of stop words from NLTK English dictionary
stop_words = set(stopwords.words('english'))

# Take a look at the stop words
print(stop_words)

# There are 179 stop words in total
print(len(stop_words))

{'of', 's', 'theirs', 'there', 'that', 't', 'should', 'does', 'are', 'over', 'then', 'here', "she's", 'through', 'between', 're', 'in', 'o', 'or', 'during', 'both', 'no', 'which', 'below', 'had', "didn't", 'further', 'own', 'me', 'his', 'about', 'can', 'after', 'll', 'am', "you're", 'most', 'up', 'against', "you've", 'ourselves', 'nor', 'these', 'too', 'at', 'yourselves', "you'd", 'has', 'i', 'be', 'once', 'hers', "wasn't", 'whom', 'didn', 'being', 'been', "aren't", 'it', 'do', 'them', "couldn't", 'having', 'doesn', 'shan', 'from', 'wouldn', 'if', 'when', 'so', 'mustn', 'to', 'itself', 'who', "wouldn't", 'hadn', 'why', 'an', 'all', "doesn't", 'and', 'any', "shouldn't", 'mightn', 'y', 'but', 'hasn', 'd', 'wasn', 'him', 'than', 'out', 'herself', "that'll", 'doing', 'under', 'those', 'she', 'were', 'for', 'other', 'off', "should've", "mightn't", 'needn', "hasn't", 'won', 'more', 'just', "hadn't", 'with', 'ours', "won't", 'while', 'yours', 'ain', 'have', 'yourself', 'a', 'before', 'their',

In [22]:

# Split the data into 90% train and 10% test
X_train, X_test, y_train, y_test = train_test_split(
  df['text'], df['label'], test_size = 0.1)
X_train.shape, X_test.shape

((39548,), (4395,))

In [23]:

# Print a single example from the train set
pd.DataFrame(X_train).iloc[0]
pd.DataFrame(y_train).iloc[0]

text    We all know that we have scarce resources! we must be aware that our resources will hastily deplete because of climate change  #NowPH
Name: 39355, dtype: object

label    2
Name: 39355, dtype: int64

#Lemmatization vs. Stemming
\

Lemmatization in linguistics is the process of grouping together the inflected forms of a word so they can be analysed as a single item, identified by the word's lemma, or dictionary form.
\

Lemmatisation is closely related to stemming. The difference is that a stemmer operates on a single word without knowledge of the context, and therefore cannot discriminate between words which have different meanings depending on part of speech. However, stemmers are typically easier to implement and run faster.

Examples:

The word "better" has "good" as its lemma. This link is missed by stemming, as it requires a dictionary look-up.

The word "walk" is the base form for the word "walking", and hence this is matched in both stemming and lemmatisation.

The word "meeting" can be either the base form of a noun or a form of a verb ("to meet") depending on the context; e.g., "in our last meeting" or "We are meeting again tomorrow". Unlike stemming, lemmatisation attempts to select the correct lemma depending on the context.

In [24]:
# Initiate the NLTK word lemmatizer
lemmatizer = nltk.stem.WordNetLemmatizer()

# Take a look at an example of lemmatization
lemmatizer.lemmatize('hours')

'hour'

In [25]:
# As an alternative to lemmatization, we could use stemming instead
stemmer = nltk.stem.SnowballStemmer("english")

# Take a look at an example of stemming
stemmer.stem("having")

'have'

In [26]:
# Create a function to clean text data
def preprocessor(text):

  # Remove all html markup from a tweet
  text = re.sub('<[^>]*>', '', text)

  # Remove @username from a tweet
  text = re.sub(r"@[^\s]+",'', text)

  # Remove http links from a tweet
  text = re.sub('http[^\s]+', '', text)

  # Find all emoticons and store them temporarily
  emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)

  # Remove all non-word characters, make all words to lower case, and add back the stored emoticons
  text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
  return text

In [27]:
# Apply the above preprocessor to clean the tweet texts
X_train = X_train.apply(preprocessor)
X_test = X_test.apply(preprocessor)

# Take a look at the first five tweets
pd.DataFrame(X_train[:5])

,text
39355,we all know that we have scarce resources we must be aware that our resources will hastily deplete because of climate change nowph
9319,good for the dept of energy u s energy department balks at trump request for names on climate change via
33568,our fight against climate change is failing one technology can change that
4879,rt report says australia can expect to have more intense thunderstorms in years ahead because of climate change
33862,rt this is worrying given it comes despite the push for renewables mitigating climate change etc time for action not words â


In [28]:
# Create a function to tokenize and lemmatize text
def tokenizer_lemmetizer(text):

  # Tokenize a tweet
  text = tokenizer.tokenize(text)

  # Remove stop words and convert a tweet to lower case
  text = [token for token in text if token not in stop_words]

  # Stem each token and combine them into a single string
  return ' '.join([lemmatizer.lemmatize(word) for word in text])

In [29]:
# Apply the above function on the train and test sets
X_train = X_train.apply(tokenizer_lemmetizer)
X_test = X_test.apply(tokenizer_lemmetizer)

# Take a look at the first five tweets
pd.DataFrame(X_train[:5])

,text
39355,know scarce resource must aware resource hastily deplete climate change nowph
9319,good dept energy u energy department balk trump request name climate change via
33568,fight climate change failing one technology change
4879,rt report say australia expect intense thunderstorm year ahead climate change
33862,rt worrying given come despite push renewables mitigating climate change etc time action word â


In [30]:
# Construct the vocabulary of the bag-of-words model
count = CountVectorizer(
  # Remove stop words
  stop_words = 'english',
  # Create 1-gram vocabulary (i.e., a single word)
  # Note: use (1, 2) to create 2-gram vocabulary
  ngram_range = (1, 1),
  # Build a vocabulary of 10000 most frequent words
  max_features = 10000)

In [31]:
# Fit and transform the train set into sparse feature vectors
X_train_bag = count.fit_transform(X_train)
print(X_train_bag.shape)

# Transform the test set into sparse feature vectors
X_test_bag = count.transform(X_test)
print(X_test_bag.shape)

(39548, 10000)
(4395, 10000)


In [32]:
# Show the library of vocabulary
print(len(count.vocabulary_))
print(count.vocabulary_)

10000
{'know': 5124, 'resource': 7528, 'aware': 948, 'hastily': 4108, 'climate': 1888, 'change': 1744, 'nowph': 6090, 'good': 3887, 'dept': 2611, 'energy': 3128, 'department': 2597, 'balk': 991, 'trump': 9083, 'request': 7503, 'fight': 3494, 'failing': 3387, 'technology': 8772, 'rt': 7686, 'report': 7488, 'say': 7811, 'australia': 918, 'expect': 3303, 'intense': 4826, 'thunderstorm': 8898, 'year': 9853, 'ahead': 539, 'worrying': 9784, 'given': 3839, 'come': 2006, 'despite': 2635, 'push': 7164, 'renewables': 7470, 'mitigating': 5749, 'time': 8921, 'action': 408, 'word': 9766, 'rare': 7261, 'china': 1803, 'criticizes': 2326, 'plan': 6748, 'exit': 3296, 'pact': 6427, 'gt': 3989, 'haunting': 4121, 'photo': 6686, 'global': 3850, 'warming': 9568, 'irma': 4923, 'wake': 9540, 'denying': 2595, 'republican': 7501, 'greedy': 3955, 'ideology': 4504, 'line': 5314, 'senior': 8006, 'diplomat': 2708, 'beijing': 1081, 'embassy': 3069, 'resigns': 7518, 'decision': 2483, 'planet': 6750, 'hottest': 4372, 

#Term Frequency Inverse Document Frequency

Term Frequency - Inverse Document Frequency (TF-IDF) is a widely used statistical method in natural language processing and information retrieval. It measures how important a term is within a document relative to a collection of documents (i.e., relative to a corpus).


Term Frequency: TF of a term or word is the number of times the term appears in a document compared to the total number of words in the document.


Inverse Document Frequency: IDF of a term reflects the proportion of documents in the corpus that contain the term. Words unique to a small percentage of documents (e.g., technical jargon terms) receive higher importance values than words common across all documents (e.g., a, the, and).


The TF-IDF of a term is calculated by multiplying TF and IDF scores.

In [33]:

# Take the raw term frequencies built by CountVectorizer as input and
# transform them into the term frequency-inverse document frequency (tf-idf)
tfidf = TfidfTransformer(use_idf = True, norm = 'l2', smooth_idf = True)
X_train_tfidf = tfidf.fit_transform(X_train_bag)
X_test_tfidf = tfidf.transform(X_test_bag)

In [34]:
print(X_train_tfidf.toarray())
print('\n')
X_train_tfidf.toarray().shape

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]




(39548, 10000)

In [35]:
# Build a logistic model
log_tfidf = LogisticRegression(solver = 'liblinear', random_state = 42)
score = log_tfidf.fit(X_train_tfidf, y_train).score(X_test_tfidf, y_test)
print(score)

0.6926052332195677


In [36]:
# Compare the model predictions to the baseline of using dummy classifier
dummy_classifier = DummyClassifier(strategy = 'stratified')
dummy_classifier.fit(X_train_tfidf, y_train).score(X_test_tfidf, y_test)

0.3560864618885097

In [37]:
# Initiate the synonym augmentation
aug_syn = naw.SynonymAug(
  aug_src = 'wordnet',
  aug_max = 3)

In [39]:
import torch
print(torch.cuda.is_available())


False


In [40]:
## Initiate the contextual word embeddings (BERT, DistilBERT, RoBERTA or XLNet) augmentation
aug_emb = naw.ContextualWordEmbsAug(
  # Other models include 'distilbert-base-uncased', 'roberta-base', etc.
  model_path = 'roberta-base',
  # You can also choose "insert"
  action = "substitute")

In [43]:
# Initiate the back translation augmentation
aug_bt = naw.BackTranslationAug(
  # Translate English to German
  from_model_name = 'facebook/wmt19-en-de',
  # Translate German back to English
  to_model_name = 'facebook/wmt19-de-en',
  # # Use GPU
  # device = 'cuda'
)

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

Some weights of FSMTForConditionalGeneration were not initialized from the model checkpoint at facebook/wmt19-en-de and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

...

vocab-src.json:   0%|          | 0.00/849k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/315k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

vocab-src.json:   0%|          | 0.00/849k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/315k [00:00<?, ?B/s]

In [41]:
# Create a function to evaluate text augmentation on model performance on test set
def evaluate_aug(aug_strategy, n, X_train, y_train, X_test, y_test):

  # Create two lists to store augmented tweets and their correponding labels
  augmented_tweets = []
  augmented_tweets_labels = []

  # Loop through the train set to create augmented tweets
  # Note: We create n augmented tweets per original tweet.
  for i in X_train.index:
    if aug_strategy == 'synonym':
      lst_augment = aug_syn.augment(X_train[i], n = n)
    elif aug_strategy == 'embedding':
      lst_augment = aug_emb.augment(X_train[i], n = n)
    else:
      lst_augment = aug_bt.augment(X_train[i], n = n)
    for augment in lst_augment:
      augmented_tweets.append(augment)
      augmented_tweets_labels.append(y_train[i])

  # Append the augmented tweets to the original tweets in the train set
  X_train_appended = X_train.append(
    pd.Series(augmented_tweets), ignore_index = True)
  y_train_appended = y_train.append(
    pd.Series(augmented_tweets_labels), ignore_index = True)

  # Apply the preprocessor to clean the tweet texts
  X_train_appended = X_train_appended.apply(preprocessor)

  # Apply tokenization and lemmetization
  X_train_appended = X_train_appended.apply(tokenizer_lemmetizer)

  # Fit and transform the appended train set into sparse feature vectors
  X_train_appended_bag = count.fit_transform(X_train_appended)

  # Transform the test set into sparse feature vectors
  X_test_bag = count.transform(X_test)

  # Take the raw term frequencies built by CountVectorizer as input and
  # transform them into the term frequency-inverse document frequency (tf-idf)
  X_train_appended_tfidf = tfidf.fit_transform(X_train_appended_bag)
  X_test_tfidf = tfidf.transform(X_test_bag)

  # Remove all elements from the lists
  augmented_tweets.clear()
  augmented_tweets_labels.clear()

  # Fit a logistic regression
  return (log_tfidf.fit(X_train_appended_tfidf, y_train_appended).
          score(X_test_tfidf, y_test))

In [44]:
# Evaluate the synonym text augmentation
score_synonym = evaluate_aug(
  aug_strategy = 'synonym',
  n = 1,
  X_train = X_train,
  y_train = y_train,
  X_test = X_test,
  y_test = y_test)
print(score_synonym)

AttributeError: 'Series' object has no attribute 'append'

In [ ]:
# Evaluate the embedding text augmentation (less than 1 hour)
score_emb = evaluate_aug(
  aug_strategy = 'embedding',
  n = 1,
  X_train = X_train,
  y_train = y_train,
  X_test = X_test,
  y_test = y_test)
print(score_emb)

In [ ]:
# Evaluate the back translation text augmentation (~10 hours)
score_bt = evaluate_aug(
  aug_strategy = 'backtranslation',
  n = 1,
  X_train = X_train,
  y_train = y_train,
  X_test = X_test,
  y_test = y_test)
print(score_bt)